Extract patient_id

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# change to your folder path
# os.chdir('C:/Users/xiao-zy19/Desktop/Johns Hopkins/Biomedical Data Design/EICU Database/eicu-collaborative-research-database-demo-2.0.1')
os.chdir('/Users/xiao-zy19/Desktop/Johns Hopkins/Biomedical Data Design/EICU/eicu-collaborative-research-database-2.0')

# import diagnosis.csv
df_diagnosis = pd.read_csv('diagnosis.csv')
df_diagnosis.sort_values(by=['patientunitstayid', 'diagnosisoffset'], inplace=True)

# select cardiovascular patients
df_cardiovascular = df_diagnosis[df_diagnosis['diagnosisstring'].str.contains('cardiovascular')]

# get shock patient 
shock_patient = df_cardiovascular[df_cardiovascular['diagnosisstring'].str.contains('shock')]

# get ventricular patient 
ventricular_patient = df_cardiovascular[df_cardiovascular['diagnosisstring'].str.contains('ventricular')]

# get chest pain patient 
chest_pain_patient = df_cardiovascular[df_cardiovascular['diagnosisstring'].str.contains('chest pain')]

# get arrhythmias patient 
arrhythmias_patient = df_cardiovascular[df_cardiovascular['diagnosisstring'].str.contains('arrhythmias')]

# put id together
df_wanted = pd.concat([shock_patient, ventricular_patient, chest_pain_patient, arrhythmias_patient])

# Get the patient ids from df_wanted & sort the patient id
# patient_id_all multiple entry patient's stayid
patient_id_all = df_wanted['patientunitstayid'].unique()
patient_id_all.sort()
print(patient_id_all)

# import patient.csv
df_patient = pd.read_csv('patient.csv')
df_patient.sort_values(by=['patientunitstayid'], inplace=True)
df_patient_buf = df_patient[df_patient['patientunitstayid'].isin(patient_id_all)]
df_1time_patient = df_patient_buf[df_patient_buf['unitvisitnumber']==1]
# print(df_1time_patient)

# select the patient id from df_1time_patient
patient_id = df_1time_patient['patientunitstayid'].unique()
print(f'Total number of patients: {len(patient_id)}')

[ 141168  141203  141227 ... 3353216 3353235 3353251]
Total number of patients: 71353


1. Heart rate

In [2]:
# import vitalPeriodic.csv & nurseCharting.csv
df_vitalPeriodic = pd.read_csv('vitalPeriodic.csv')
df_vitalPeriodic.sort_values(by=['patientunitstayid', 'observationoffset'], inplace=True)
df_nurseCharting = pd.read_csv('nurseCharting.csv')
df_nurseCharting.sort_values(by=['patientunitstayid', 'nursingchartoffset'], inplace=True)


# select the wanted patient
df_vitalPeriodic = df_vitalPeriodic[df_vitalPeriodic['patientunitstayid'].isin(patient_id)]
df_nurseCharting = df_nurseCharting[df_nurseCharting['patientunitstayid'].isin(patient_id)]

# save the wanted file (uncomment the code to save)
# df_vitalPeriodic.to_csv('vitalPeriodic_wanted.csv', index=False)

# print the shape of the wanted file
print(f'vitalperiodic shape: {df_vitalPeriodic.shape}')
print(f'nurseCharting shape: {df_nurseCharting.shape}')

# nursingchartcelltypevallabel Heart Rate
df_nurseCharting = df_nurseCharting[df_nurseCharting['nursingchartcelltypevallabel']=='Heart Rate']
df_nurseCharting = df_nurseCharting.rename(columns={'nursingchartoffset': 'observationoffset', 'nursingchartvalue':'heartrate'})
print(df_nurseCharting.head())

# define heartrate preprocessing function
def normal_heartrate(num):
    """
    Function to normalize heart rate values.

    Parameters:
        num: the originial input value
    Return:
        num: the normalized output value
    """
    # Return null values direcly
    if pd.isna(num):
        return num
    # Remove values out of range
    elif num > 300 or num < 0:
        return np.nan
    # Return normal values directly
    else:
        return num

# extract heart rate from df_vitalPeriodic & df_nurseCharting
HR_v = df_vitalPeriodic[['patientunitstayid', 'observationoffset', 'heartrate']]
HR_n = df_nurseCharting[['patientunitstayid', 'observationoffset', 'heartrate']]
HR = pd.concat([HR_v, HR_n]).astype(float)
HR.sort_values(by=['patientunitstayid', 'observationoffset'], inplace=True)
print(f'First 5 rows of HR: \n{HR.head()} \n')

# exclude abnormal heart rate values
HR.loc[:, 'heartrate'] = HR['heartrate'].apply(normal_heartrate)

# save HR to csv file (uncomment the code to save)
# HR.to_csv('HR.csv', index=False)

value_position_dict = {}
first_occurrences = []
for idx, value in enumerate(HR['patientunitstayid']):
    # if the value is not in the dictionary, add it and create index
    if value not in value_position_dict:
        value_position_dict[value] = idx
        first_occurrences.append(idx)

first_occurrences.append(len(HR))
# create first occurrence index for every patient
HR_index = pd.Series(first_occurrences)
print(f'shape of HR_index: {HR_index.shape}')
print(f'First 5 rows of HR_index: \n{HR_index.head()}')

# double check the index is correct
# print(HR.iloc[HR_index].head())
# print(HR.iloc[[156, 157, 158, 159]])
# print(HR.iloc[[1015, 1016, 1017, 1018]])

vitalperiodic shape: (63195275, 19)
nurseCharting shape: (61929777, 8)
           nursingchartid  patientunitstayid  observationoffset  \
151470706       164546008             141168                  6   
151470614       146409413             141168                 21   
151470622       146455198             141168                 36   
151470649       201018075             141168                 51   
151470709       200253204             141168                 66   

           nursingchartentryoffset nursingchartcelltypecat  \
151470706                        6             Vital Signs   
151470614                       21             Vital Signs   
151470622                       36             Vital Signs   
151470649                       51             Vital Signs   
151470709                       66             Vital Signs   

          nursingchartcelltypevallabel nursingchartcelltypevalname heartrate  
151470706                   Heart Rate                  Heart Rate       1

In [3]:
# 第一次进入ICU的心血管病疾病患者ID集合
first_time_icu_patients = set(patient_id)
print(f'Number of first time ICU patients: {len(first_time_icu_patients)}')

# 有heartrate数据的患者ID集合
patients_with_heartrate = set(HR['patientunitstayid'].unique())
print(f'Number of patients with heart rate data: {len(patients_with_heartrate)}')

# 没有heartrate数据的患者ID集合
# patients_without_heartrate = first_time_icu_patients - patients_with_heartrate
# print(f'Number of patients without heart rate data: {len(patients_without_heartrate)}')

Number of first time ICU patients: 71353
Number of patients with heart rate data: 71242


2. Temperature

In [4]:
# import vitalPeriodic.csv & nurseCharting.csv
df_vitalPeriodic = pd.read_csv('vitalPeriodic.csv')
df_vitalPeriodic.sort_values(by=['patientunitstayid', 'observationoffset'], inplace=True)
df_nurseCharting = pd.read_csv('nurseCharting.csv')
df_nurseCharting.sort_values(by=['patientunitstayid', 'nursingchartoffset'], inplace=True)


# select the wanted patient
df_vitalPeriodic = df_vitalPeriodic[df_vitalPeriodic['patientunitstayid'].isin(patient_id)]
df_nurseCharting = df_nurseCharting[df_nurseCharting['patientunitstayid'].isin(patient_id)]

# save the wanted file (uncomment the code to save)
# df_vitalPeriodic.to_csv('vitalPeriodic_wanted.csv', index=False)

# print the shape of the wanted file
print(f'vitalperiodic shape: {df_vitalPeriodic.shape}')
print(f'nurseCharting shape: {df_nurseCharting.shape}')
# define temperature transformation function
def normal_temperature(num):
    """
    Function to normalize temperature values.

    Parameters:
        num: the originial input value
    Return:
        num: the normalized output value
    """
    # Return null values direcly
    if pd.isna(num):
        return num
    # Convert Fahrenheit to Celsius
    # And apply the function again
    elif num > 50:
        return normal_temperature((num - 32) * 5 / 9)
    # Remove values out of range
    elif num < 15 or num > 45:
        return np.nan
    # Return normal values directly
    else:
        return num
# nursingchartcelltypevallabel Temperature
df_nurseCharting = df_nurseCharting[df_nurseCharting['nursingchartcelltypevallabel']=='Temperature']
df_nurseCharting = df_nurseCharting.rename(columns={'nursingchartoffset': 'observationoffset', 'nursingchartvalue':'temperature'})
print(df_nurseCharting.head())

# extract temperature from df_vitalPeriodic & df_nurseCharting
Temp_v = df_vitalPeriodic[['patientunitstayid', 'observationoffset', 'temperature']]
Temp_n = df_nurseCharting[['patientunitstayid', 'observationoffset', 'temperature']]

# delete the rows with string values
Temp_n = Temp_n[Temp_n['temperature'].apply(lambda x: str(x).replace('.', '', 1).isdigit())]
print(f'First 5 rows of Temp_n: \n{Temp_n.head()} \n')

Temp = pd.concat([Temp_v, Temp_n]).astype(float)
Temp.sort_values(by=['patientunitstayid', 'observationoffset'], inplace=True)
print(f'First 5 rows of Temp: \n{Temp.head()} \n')

# exclude abnormal heart rate values
Temp.loc[:, 'temperature'] = Temp['temperature'].apply(normal_temperature)

# save Temp to csv file (uncomment the code to save)
# Temp.to_csv('Temp.csv', index=False)

value_position_dict = {}
first_occurrences = []
for idx, value in enumerate(Temp['patientunitstayid']):
    # if the value is not in the dictionary, add it and create index
    if value not in value_position_dict:
        value_position_dict[value] = idx
        first_occurrences.append(idx)

first_occurrences.append(len(Temp))
# create first occurrence index for every patient
Temp_index = pd.Series(first_occurrences)
print(f'First 5 rows of Temp_index: \n{Temp_index.head()}')

vitalperiodic shape: (63195275, 19)
nurseCharting shape: (61929777, 8)
           nursingchartid  patientunitstayid  observationoffset  \
151470743       222080627             141168                351   
151470759       222080625             141168                351   
151470766       222080626             141168                351   
151470826       257836794             141168                563   
151470832       257836793             141168                563   

           nursingchartentryoffset nursingchartcelltypecat  \
151470743                      351             Vital Signs   
151470759                      351             Vital Signs   
151470766                      351             Vital Signs   
151470826                      563             Vital Signs   
151470832                      563             Vital Signs   

          nursingchartcelltypevallabel nursingchartcelltypevalname  \
151470743                  Temperature             Temperature (F)   
151470759    

In [5]:
# 第一次进入ICU的心血管疾病患者ID集合
first_time_icu_patients = set(patient_id)
print(f'First time ICU patients: {len(first_time_icu_patients)}')

# 有温度数据的患者ID集合
patients_with_temperature = set(Temp['patientunitstayid'].unique())
print(f'Patients with temperature: {len(patients_with_temperature)}')

# 没有温度数据的患者ID集合
# patients_without_temperature = first_time_icu_patients - patients_with_temperature
# print(f'Patients without temperature: {len(patients_without_temperature)}')
# print(f'Patients without temperature: \n {patients_without_temperature}')

First time ICU patients: 71353
Patients with temperature: 71263


3. Blood pressure

In [6]:
# import vitalPeriodic.csv & nurseCharting.csv
df_vitalPeriodic = pd.read_csv("vitalPeriodic.csv")
df_vitalPeriodic.sort_values(
    by=["patientunitstayid", "observationoffset"], inplace=True
)
df_nurseCharting = pd.read_csv("nurseCharting.csv")
df_nurseCharting.sort_values(
    by=["patientunitstayid", "nursingchartoffset"], inplace=True
)
df_vitalAPeriodic = pd.read_csv("vitalAperiodic.csv")
df_vitalAPeriodic.sort_values(
    by=["patientunitstayid", "observationoffset"], inplace=True
)


# select the wanted patient
df_vitalPeriodic = df_vitalPeriodic[
    df_vitalPeriodic["patientunitstayid"].isin(patient_id)
]
df_nurseCharting = df_nurseCharting[
    df_nurseCharting["patientunitstayid"].isin(patient_id)
]
df_vitalAPeriodic = df_vitalAPeriodic[
    df_vitalAPeriodic["patientunitstayid"].isin(patient_id)
]

# save the wanted file (uncomment the code to save)
# df_vitalPeriodic.to_csv('vitalPeriodic_wanted.csv', index=False)
# df_nurseCharting.to_csv('nurseCharting_wanted.csv', index=False)
# df_vitalAPeriodic.to_csv('vitalAPeriodic_wanted.csv', index=False)

# print the shape of the wanted file
print(f"vitalperiodic shape: {df_vitalPeriodic.shape}")
print(f"nurseCharting shape: {df_nurseCharting.shape}")
print(f"vitalAPeriodic shape: {df_vitalAPeriodic.shape}")

# nursingchartcelltypevallabel Non-Invasive BP Systolic
df_nurseCharting_noninvasive = df_nurseCharting[
    df_nurseCharting["nursingchartcelltypevalname"] == "Non-Invasive BP Systolic"
]
df_nurseCharting_noninvasive = df_nurseCharting_noninvasive.rename(
    columns={
        "nursingchartoffset": "observationoffset",
        "nursingchartvalue": "Non-Invasive BP Systolic",
    }
)
# print(df_nurseCharting_noninvasive.head())

# nursingchartcelltypevallabel Invasive BP Systolic
df_nurseCharting_invasive = df_nurseCharting[
    df_nurseCharting["nursingchartcelltypevalname"] == "Invasive BP Systolic"
]
df_nurseCharting_invasive = df_nurseCharting_invasive.rename(
    columns={
        "nursingchartoffset": "observationoffset",
        "nursingchartvalue": "Invasive BP Systolic",
    }
)
# print(df_nurseCharting_invasive.head())

# extract systolics from vitalPeriodic, nurseCharting & vitalAPeriodic
systemicsystolic = df_vitalPeriodic[
    ["patientunitstayid", "observationoffset", "systemicsystolic"]
]
non_invasive_BP_Systolic = df_nurseCharting_noninvasive[
    ["patientunitstayid", "observationoffset", "Non-Invasive BP Systolic"]
]
invasive_BP_Systolic = df_nurseCharting_invasive[
    ["patientunitstayid", "observationoffset", "Invasive BP Systolic"]
]
Noninvasivesystolic = df_vitalAPeriodic[
    ["patientunitstayid", "observationoffset", "noninvasivesystolic"]
]


def create_index(df, column_name="patientunitstayid"):
    value_position_dict = {}
    first_occurrences = []

    for idx, value in enumerate(df[column_name]):
        if value not in value_position_dict:
            value_position_dict[value] = idx
            first_occurrences.append(idx)

    first_occurrences.append(len(df))
    return pd.Series(first_occurrences)


# create index for each variable
systemicsystolic_index = create_index(systemicsystolic)
non_invasive_BP_Systolic_index = create_index(non_invasive_BP_Systolic)
invasive_BP_Systolic_index = create_index(invasive_BP_Systolic)
Noninvasivesystolic_index = create_index(Noninvasivesystolic)

vitalperiodic shape: (63195275, 19)
nurseCharting shape: (61929777, 8)
vitalAPeriodic shape: (11017182, 13)


In [7]:
# 第一次进入ICU的心血管疾病患者ID集合
first_time_icu_patients = set(patient_id)
print(f'First time ICU patients: {len(first_time_icu_patients)}')

# 有systemicsystolic noninvasivesystolic invasive BP Systolic non_invasive_BP Systolic的患者ID集合
patients_with_systolic = set(systemicsystolic["patientunitstayid"].unique())
patients_with_noninvasive = set(non_invasive_BP_Systolic["patientunitstayid"].unique())
patients_with_invasive = set(invasive_BP_Systolic["patientunitstayid"].unique())
patients_with_Noninvasive = set(Noninvasivesystolic["patientunitstayid"].unique())

# 求并集
patients_with_bp = patients_with_systolic.union(patients_with_noninvasive, patients_with_invasive, patients_with_Noninvasive)

# 求交集长度
print(f'Patients with all systolic: {len(patients_with_bp)}')

First time ICU patients: 71353
Patients with all systolic: 71238


4. Glasgow Coma Scale

In [8]:
# import nurseCharting.csv
df_nurseCharting = pd.read_csv('nurseCharting.csv')
df_nurseCharting.sort_values(by=['patientunitstayid', 'nursingchartoffset'], inplace=True)

# select the wanted patient
df_nurseCharting = df_nurseCharting[df_nurseCharting['patientunitstayid'].isin(patient_id)]

# print the shape of the wanted file
print(f'nurseCharting shape: {df_nurseCharting.shape}')

# nursingchartcelltypevallabel Temperature
df_nurseCharting = df_nurseCharting[df_nurseCharting['nursingchartcelltypevallabel']=='Glasgow coma score']
df_nurseCharting = df_nurseCharting.rename(columns={'nursingchartoffset': 'observationoffset', 'nursingchartvalue':'Glasgow score'})
print(df_nurseCharting.head())
print(df_nurseCharting.shape)

# extract glasgow score from df_nurseCharting
Glasgow = df_nurseCharting[['patientunitstayid', 'observationoffset', 'Glasgow score']].copy()
Glasgow.sort_values(by=['patientunitstayid', 'observationoffset'], inplace=True)
print(f'First 5 rows of Temp: \n{Glasgow.head()} \n')

value_position_dict = {}
first_occurrences = []
for idx, value in enumerate(Glasgow['patientunitstayid']):
    # if the value is not in the dictionary, add it and create index
    if value not in value_position_dict:
        value_position_dict[value] = idx
        first_occurrences.append(idx)

first_occurrences.append(len(Glasgow))
# create first occurrence index for every patient
Glasgow_index = pd.Series(first_occurrences)
print(f'First 5 rows of Temp_index: \n{Glasgow_index.head()}')

nurseCharting shape: (61929777, 8)
           nursingchartid  patientunitstayid  observationoffset  \
151475922       247094422             141227              -2230   
151475370       283246555             141227               -663   
151475782       174895080             141227               -132   
151475638       265182694             141227                 57   
151475246       174895188             141227                322   

           nursingchartentryoffset nursingchartcelltypecat  \
151475922                    -2230                  Scores   
151475370                     -663                  Scores   
151475782                     -132                  Scores   
151475638                       57                  Scores   
151475246                      322                  Scores   

          nursingchartcelltypevallabel nursingchartcelltypevalname  \
151475922           Glasgow coma score                   GCS Total   
151475370           Glasgow coma score           

In [9]:
# 第一次进入ICU的心血管疾病患者ID集合
first_time_icu_patients = set(patient_id)
print(f'First time ICU patients: {len(first_time_icu_patients)}')

# 有格拉斯哥昏迷评分数据的患者ID集合
patients_with_glasgow = set(Glasgow['patientunitstayid'].unique())
print(f'Patients with glasgow: {len(patients_with_glasgow)}')

# 没有格拉斯哥昏迷评分数据的患者ID集合
# patients_without_glasgow = first_time_icu_patients - patients_with_glasgow
# print(f'Patients without glasgow: {len(patients_without_glasgow)}')
# print(patients_without_glasgow)


First time ICU patients: 71353
Patients with glasgow: 52429


5. lab results

In [10]:
# import lab.csv
df_lab = pd.read_csv('lab.csv')
df_lab.sort_values(by=['patientunitstayid', 'labresultoffset'], inplace=True)

# select the wanted patient
df_lab = df_lab[df_lab['patientunitstayid'].isin(patient_id)]

# print the shape of the wanted file
print(f'lab shape: {df_lab.shape}')

def create_index(df):
    """
    Create first occurrence index for every patient
    """
    # create a dictionary to store the first occurrence index
    value_position_dict = {}
    first_occurrences = []
    for idx, value in enumerate(df['patientunitstayid']):
        # if the value is not in the dictionary, add it and create index
        if value not in value_position_dict:
            value_position_dict[value] = idx
            first_occurrences.append(idx)

    first_occurrences.append(len(df))
    # create first occurrence index for every patient
    df_index = pd.Series(first_occurrences)
    return df_index

# lab BUN
df_lab_BUN = df_lab[df_lab['labname']=='BUN']
df_lab_BUN = df_lab_BUN.rename(columns={'labresultoffset': 'observationoffset', 'labresult':'BUN'})

# lab WBC x 1000
df_lab_WBC = df_lab[df_lab['labname']=='WBC x 1000']
df_lab_WBC = df_lab_WBC.rename(columns={'labresultoffset': 'observationoffset', 'labresult':'WBC x 1000'})

# lab bicarbonate
df_lab_bicarbonate = df_lab[df_lab['labname']=='bicarbonate']
df_lab_bicarbonate = df_lab_bicarbonate.rename(columns={'labresultoffset': 'observationoffset', 'labresult':'bicarbonate'})

# lab sodium
df_lab_sodium = df_lab[df_lab['labname']=='sodium']
df_lab_sodium = df_lab_sodium.rename(columns={'labresultoffset': 'observationoffset', 'labresult':'sodium'})

# lab potassium
df_lab_potassium = df_lab[df_lab['labname']=='potassium']
df_lab_potassium = df_lab_potassium.rename(columns={'labresultoffset': 'observationoffset', 'labresult':'potassium'})

# lab total bilirubin
df_lab_bilirubin = df_lab[df_lab['labname']=='total bilirubin']
df_lab_bilirubin = df_lab_bilirubin.rename(columns={'labresultoffset': 'observationoffset', 'labresult':'total bilirubin'})

# extract lab results from lab.csv
BUN = df_lab_BUN[['patientunitstayid', 'observationoffset', 'BUN']].copy()
WBC = df_lab_WBC[['patientunitstayid', 'observationoffset', 'WBC x 1000']].copy()
bicarbonate = df_lab_bicarbonate[['patientunitstayid', 'observationoffset', 'bicarbonate']].copy()
sodium = df_lab_sodium[['patientunitstayid', 'observationoffset', 'sodium']].copy()
potassium = df_lab_potassium[['patientunitstayid', 'observationoffset', 'potassium']].copy()
bilirubin = df_lab_bilirubin[['patientunitstayid', 'observationoffset', 'total bilirubin']].copy()

# sort the lab results by patient id and observationoffset
BUN.sort_values(by=['patientunitstayid', 'observationoffset'], inplace=True)
WBC.sort_values(by=['patientunitstayid', 'observationoffset'], inplace=True)
bicarbonate.sort_values(by=['patientunitstayid', 'observationoffset'], inplace=True)
sodium.sort_values(by=['patientunitstayid', 'observationoffset'], inplace=True)
potassium.sort_values(by=['patientunitstayid', 'observationoffset'], inplace=True)
bilirubin.sort_values(by=['patientunitstayid', 'observationoffset'], inplace=True)

# print the head of the lab results
# print(f'First 5 rows of lab_BUN: \n{BUN.head()} \n')
# print(f'First 5 rows of lab_WBC: \n{WBC.head()} \n')
# print(f'First 5 rows of lab_bicarbonate: \n{bicarbonate.head()} \n')
# print(f'First 5 rows of lab_sodium: \n{sodium.head()} \n')
# print(f'First 5 rows of lab_potassium: \n{potassium.head()} \n')
print(f'First 5 rows of lab_bilirubin: \n{bilirubin.head()} \n')

# create index for each variable
BUN_index = create_index(BUN)
WBC_index = create_index(WBC)
bicarbonate_index = create_index(bicarbonate)
sodium_index = create_index(sodium)
potassium_index = create_index(potassium)
bilirubin_index = create_index(bilirubin)

lab shape: (16828353, 10)
First 5 rows of lab_bilirubin: 
     patientunitstayid  observationoffset  total bilirubin
102             141168                516              2.6
104             141168               1133              4.1
66              141168               2026              5.2
758             141203              -1580              0.4
747             141203               -473              0.4 



In [11]:
# 第一次进入ICU患者ID集合
first_time_icu_patients = set(patient_id)
print(f'First time ICU patients: {len(first_time_icu_patients)}')

# 包含BUN WBC bicarbonate sodium potassium bilirubin的患者ID集合
BUN_patients = set(BUN['patientunitstayid'])
WBC_patients = set(WBC['patientunitstayid'])
bicarbonate_patients = set(bicarbonate['patientunitstayid'])
sodium_patients = set(sodium['patientunitstayid'])
potassium_patients = set(potassium['patientunitstayid'])
bilirubin_patients = set(bilirubin['patientunitstayid'])

# 求交集
lab_patients = BUN_patients & WBC_patients & bicarbonate_patients & sodium_patients & potassium_patients & bilirubin_patients
print(f'patients with all lab results: {len(lab_patients)}')

First time ICU patients: 71353
patients with all lab results: 52197


Final: 取所有patient的交集

In [12]:
Final_available_patients = patients_with_heartrate & patients_with_temperature & patients_with_bp & patients_with_glasgow & lab_patients
print(f'Final available patients: {len(Final_available_patients)}')

Final available patients: 38756
